### Train / Test Split

In [18]:
import json
import os
import random
from sklearn.model_selection import train_test_split

In [19]:
base_dir = '/home/eunseo/AUE8088-PA2/datasets/kaist-rgbt'

In [20]:
train_txt_path = os.path.join(base_dir,'train-all-04.txt')
output_train_txt_path = os.path.join(base_dir,'train_split.txt')
output_val_txt_path = os.path.join(base_dir,'val_split.txt')

In [21]:
with open(train_txt_path, 'r') as file:
    lines = file.readlines()

train_lines, val_lines = train_test_split(lines, test_size=0.2, random_state=10)
train_lines = sorted(train_lines)
val_lines = sorted(val_lines)

In [13]:
with open(output_train_txt_path, 'w+') as file:
    for line in train_lines:
        file.write(line)
    file.seek(0)
    train_cnt = len(file.readlines())
print(f'#train_lines: {train_cnt}')

with open(output_val_txt_path, 'w+') as file:
    for line in val_lines:
        file.write(line)
    file.seek(0)
    val_cnt = len(file.readlines())
print(f'#val_lines: {val_cnt}')

#train_lines: 10030
#val_lines: 2508


### Create KAIST_annotation.json for validation subset

In [24]:
import xml.etree.ElementTree as ET

json_data = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 0, "name": "person"},
            {"id": 1, "name": "cyclist"},
            {"id": 2, "name": "people"},
            {"id": 3, "name": "person?"}
        ]
    }

name_to_id = {category["name"]: category["id"] for category in json_data["categories"]}
image_id = 0
annotation_id = 0

In [25]:
labels_xml_path = os.path.join(base_dir, 'train/labels-xml')

In [ ]:
with open(output_val_txt_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        image_path = line.strip()  
        image_filename = os.path.basename(image_path)  
        xml_filename = image_filename.replace('.jpg', '.xml') 
        xml_path = os.path.join(labels_xml_path, xml_filename)

        print(xml_path, "/t conversion to json completed")
        tree = ET.parse(xml_path)
        root = tree.getroot()

        json_data["images"].append({
            "id": image_id,
            "im_name": root.find('filename').text,
            "height": int(root.find('size/height').text),
            "width": int(root.find('size/width').text)
        })

        for obj in root.findall('object'):
            bbox = obj.find('bndbox')
            x = int(bbox.find('x').text)
            y = int(bbox.find('y').text)
            w = int(bbox.find('w').text)
            h = int(bbox.find('h').text)

            json_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": int(name_to_id.get(obj.find('name').text)), 
                "bbox": [x, y, w, h],
                "height": h,
                "occlusion": int(obj.find('occlusion').text),
                "ignore": int(obj.find('difficult').text) if obj.find('difficult') is not None else 0
            })
            annotation_id += 1

        image_id += 1


In [28]:
with open('utils/eval/KAIST_annotation.json', 'w') as outfile:
    json.dump(json_data, outfile, indent=4)

In [ ]:
# Download script/URL (optional) ---------------------------------------------------------------------------------------
download: |
  import os
  import xml.etree.ElementTree as ET
  from tqdm import tqdm
  from utils.general import download, Path
  from utils.general import Path

  def convert_label(names, lb_path, lb_path_new):
      def convert_box(size, box):
          dw, dh = 1. / size[0], 1. / size[1]
          x, y, w, h = box
          return x * dw, y * dh, w * dw, h * dh

      with open(lb_path) as in_file:
          tree = ET.parse(in_file)

      with open(lb_path_new, 'w') as out_file:
          root = tree.getroot()
          size = root.find('size')
          w = int(size.find('width').text)
          h = int(size.find('height').text)

          for obj in root.iter('object'):
              cls = obj.find('name').text
              xmlbox = obj.find('bndbox')
              bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('x', 'y', 'w', 'h')])
              occ = int(obj.find('occlusion').text)
              cls_id = names.index(cls)  # class id
              out_file.write(" ".join([str(a) for a in (cls_id, *bb, occ)]) + '\n')

  path = Path('/home/ubuntu/datasets/kaist-cvpr15-aue8088')  # dataset root dir
  with open(str(path / 'train-all-04.txt'), 'r') as fp:
  # with open(str(path / 'test-all-20.txt'), 'r') as fp:
      labels = [f.replace('{}/', '').replace('images', 'labels').replace('.jpg', '.xml').rstrip('\n') for f in fp.readlines()]

  with open(path / 'kaist-rgbt.names') as fp:
      names = [f.rstrip('\n') for f in fp.readlines()]

  for label in tqdm(labels, total=len(labels)):
      lb_file = str(path / label)
      lb_file_new = lb_file.replace('labels', 'labels_converted').replace('.xml', '.txt')

      os.makedirs(os.path.dirname(lb_file_new), exist_ok=True)

      convert_label(names, lb_file, lb_file_new)  # convert labels to YOLO format